In [20]:
import yfinance as yf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import cm
from tda_m2r import *
import pandas as pd
from ripser import ripser, Rips
import persim
import gudhi
import requests
from datetime import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [21]:
sp500 = yf.Ticker("^GSPC")
sp500 = sp500.history(period = 'max')
sp500.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0


In [22]:
# removing unecessary columns
sp500 = sp500.drop(columns = ['Dividends', 'Stock Splits'])

In [23]:
sp500_close_shifted = sp500
sp500_close_shifted['Close-1'] = sp500['Close'].shift(1)
sp500_close_shifted['Close-2'] = sp500['Close'].shift(2)
sp500_close_shifted['Close-3'] = sp500['Close'].shift(3)
sp500_close_shifted.head()

,Open,High,Low,Close,Volume,Close-1,Close-2,Close-3
Date,,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,NaN,NaN,NaN
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,17.660000,NaN,NaN
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,17.760000,17.660000,NaN
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,17.719999,17.760000,17.66
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,17.549999,17.719999,17.76


In [24]:
# use only the stock close data from last years (252 days).
sp500new = sp500_close_shifted[-252:]
std_dev = sp500new.std(ddof=1)
mu = sp500new.mean()
sp500new_std = (sp500new - mu) / std_dev
sp500new_std.head()

,Open,High,Low,Close,Volume,Close-1,Close-2,Close-3
Date,,,,,,,,
2023-06-08 00:00:00-04:00,-1.304130,-1.283275,-1.272160,-1.238300,-0.124432,-1.305610,-1.250816,-1.271990
2023-06-09 00:00:00-04:00,-1.200466,-1.212529,-1.184138,-1.224165,-0.183911,-1.229658,-1.297944,-1.242854
2023-06-12 00:00:00-04:00,-1.190612,-1.162195,-1.147729,-1.109269,0.051403,-1.215480,-1.221726,-1.290149
2023-06-13 00:00:00-04:00,-1.063745,-1.060893,-1.018586,-1.023020,0.538900,-1.100244,-1.207499,-1.213660
2023-06-14 00:00:00-04:00,-1.024559,-1.013606,-1.051518,-1.012755,0.504467,-1.013739,-1.091858,-1.199382


In [ ]:
def df_to_X_y(df, predictors, target):

    M = df[predictors].to_numpy()
    dates = df.index.to_numpy()
    X = M.reshape((len(dates), M.shape[1], 1))
    y = df[target].to_numpy()

    return dates, X.astype(np.float32), y.astype(np.float32)
